<a href="https://colab.research.google.com/github/deybvagm/invoice-object-detection/blob/master/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Training faster RCNN and SSD object detection models

This notebook train this two models using the Tensorflow Object Detection API on an invoice dataset that be found [here](https://rrc.cvc.uab.es/?ch=13). This notebook covers the following:
- Configuration of the environment by installing Tensorflow Object Detection API
- Creation of TF records based on images of invoiceswith their respective bounding boxes
- Train of the models faster RCNN and SSD

In [0]:
!pip install numpy==1.17.4

In [1]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [0]:
from sklearn.model_selection import train_test_split
from PIL import Image
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

import imutils
import os
import cv2


In [0]:
print('tf version ', tf.__version__)
print('np version ', np.__version__)

### Environment configuration

We need:
- Tensorflow object detection API: For this we are going to clone the repository
- We need to install some needed packages in the system
- We need compile object detection API

In [0]:
!git clone https://github.com/tensorflow/models.git
!git clone https://github.com/deybvagm/invoice-object-detection.git
!mv invoice-object-detection res

In [0]:
!apt-get install protobuf-compiler python-pil python-lxml python-tk
!pip install Cython

In [0]:
%cd '/content/models/'
!git checkout c9f03bf6a8ae58b9ead119796a6c3cd9bd04d450
!mv /content/res/resources/*.py '/content/models/research/object_detection/'

In [0]:
# compile model definition
%cd '/content/models/research/'
!protoc object_detection/protos/*.proto --python_out=.

In [0]:
# set the environment
os.environ['PYTHONPATH'] += ':/content/models/research/:/content/models/research/slim'

In [0]:
!python setup.py build
!python setup.py install

In [0]:
# to test if everything is ok
!python /content/models/research/object_detection/builders/model_builder_test.py

In [0]:
# create a symbolic link to avoid writting complete paths
%cd '/content/'
!ln -s '/content/models/research/object_detection/' object_detection

In [0]:
# Object detection imports
from object_detection.utils.dataset_util import bytes_list_feature
from object_detection.utils.dataset_util import float_list_feature
from object_detection.utils.dataset_util import int64_list_feature
from object_detection.utils.dataset_util import int64_feature
from object_detection.utils.dataset_util import bytes_feature
from object_detection.utils import label_map_util

### Helper functions

In order that Tensorflow could understand the data we have we need to create standard Tensorflow Records that basically have the following info:
- The image (encoded)
- Bounding boxes coordinates
- Format: JPG, PNG, etc
- Labels
- Image dimensions (width, height)

In [0]:
class TFAnnotation:
  def __init__(self):
    # initialize the bounding box + label lists
    self.xMins = []
    self.xMaxs = []
    self.yMins = []
    self.yMaxs = []
    self.textLabels = []
    self.classes = []
    self.difficult = []

    # initialize additional variables, including the image
    # itself, spatial dimensions, encoding, and filename
    self.image = None
    self.width = None
    self.height = None
    self.encoding = None
    self.filename = None

  def build(self):
    # encode the attributes using their respective TensorFlow
    # encoding function
    w = int64_feature(self.width)
    h = int64_feature(self.height)
    filename = bytes_feature(self.filename.encode("utf8"))
    encoding = bytes_feature(self.encoding.encode("utf8"))
    image = bytes_feature(self.image)
    xMins = float_list_feature(self.xMins)
    xMaxs = float_list_feature(self.xMaxs)
    yMins = float_list_feature(self.yMins)
    yMaxs = float_list_feature(self.yMaxs)
    textLabels = bytes_list_feature(self.textLabels)
    classes = int64_list_feature(self.classes)
    difficult = int64_list_feature(self.difficult)

    # construct the TensorFlow-compatible data dictionary
    data = {
        "image/height": h,
        "image/width": w,
        "image/filename": filename,
        "image/source_id": filename,
        "image/encoded": image,
        "image/format": encoding,
        "image/object/bbox/xmin": xMins,
        "image/object/bbox/xmax": xMaxs,
        "image/object/bbox/ymin": yMins,
        "image/object/bbox/ymax": yMaxs,
        "image/object/class/text": textLabels,
        "image/object/class/label": classes,
        "image/object/difficult": difficult,
    }
    return data

### Creating TF Records

In [0]:
#donwload training data from S3 (without records)
!wget https://invoiceproject.s3-us-west-2.amazonaws.com/invoice_no_records.tar.gz
!tar -zxvf invoice_no_records.tar.gz

In [0]:
class Conf():
  def __init__(self):
    self.BASE_PATH = "invoice"

    self.ANNOT_PATH = os.path.sep.join([self.BASE_PATH, "annotations.csv"])

    self.TRAIN_RECORD = os.path.sep.join([self.BASE_PATH,"records/training.record"])

    self.TEST_RECORD = os.path.sep.join([self.BASE_PATH,"records/testing.record"])

    self.CLASSES_FILE = os.path.sep.join([self.BASE_PATH,"records/classes.pbtxt"])

    self.TEST_SIZE = 0.25

    self.CLASSES = {"header": 1, "date": 2, "total": 3}

In [0]:
config = Conf()

In [18]:
%cd '/content'

/content


In [0]:
def create_records(config):  
  f = open(config.CLASSES_FILE, "w")
  for (k, v) in config.CLASSES.items():
    item = ("item {\n"
      "\tid: " + str(v) + "\n"
      "\tname: '" + k + "'\n"
      "}\n")
    f.write(item)
  f.close()
  D = {}
  rows = open(config.ANNOT_PATH).read().strip().split("\n")
  discarded_rows = 0
  discarded_labels = 0
  print('rows in file: ', len(rows))
  
  for row in rows[1:]:
    row = row.split(",")
    if len(row) < 11:
      discarded_rows += 1
      continue  

    imagePath = row[0]
    startX = float(row[6].split(":")[1].strip("\"'}"))
    startY = float(row[7].split(":")[1].strip("\"'}"))
    width = float(row[8].split(":")[1].strip("\"'}"))
    height = float(row[9].split(":")[1].strip("\"'}"))

    endX = startX + width
    endY = startY + height
    label = row[-1].split(":")[1].strip("\"'}")
    
    if label not in config.CLASSES:
      discarded_labels += 1
      continue
    
    p = os.path.sep.join([config.BASE_PATH, imagePath])
    b = D.get(p, [])

    b.append((label, (startX, startY, endX, endY)))
    D[p] = b

  print('discarded rows: ', discarded_rows)
  print('discarded labels: ', discarded_labels)
  
  (trainKeys, testKeys) = train_test_split(list(D.keys()),
  test_size=config.TEST_SIZE, random_state=42)
  
  datasets = [
    ("train", trainKeys, config.TRAIN_RECORD),
    ("test", testKeys, config.TEST_RECORD)
  ]
  
  for (dType, keys, outputPath) in datasets:    
    print("[INFO] processing '{}'...".format(dType))
    writer = tf.python_io.TFRecordWriter(outputPath)
    total = 0
    
    for k in keys:
      encoded = tf.gfile.GFile(k, "rb").read()
      encoded = bytes(encoded)      
      pilImage = Image.open(k)
      (w, h) = pilImage.size[:2]
      
      filename = k.split(os.path.sep)[-1]
      encoding = filename[filename.rfind(".") + 1:]
      
      tfAnnot = TFAnnotation()
      tfAnnot.image = encoded
      tfAnnot.encoding = encoding
      tfAnnot.filename = filename
      tfAnnot.width = w
      tfAnnot.height = h
      
      for (label, (startX, startY, endX, endY)) in D[k]:
        # TensorFlow assumes all bounding boxes are in the
        # range [0, 1] so we need to scale them
        xMin = startX / w
        xMax = endX / w
        yMin = startY / h
        yMax = endY / h

        #####JUST INSPECTION- REMOVE

        # load the input image from disk and denormalize the
        # bounding box coordinates
        #image = cv2.imread(k)
        #startX = int(xMin * w)
        #startY = int(yMin * h)
        #endX = int(xMax * w)
        #endY = int(yMax * h)

        # draw the bounding box on the image
        #cv2.namedWindow("Image", cv2.WINDOW_NORMAL)
        #cv2.rectangle(image, (startX, startY), (endX, endY),(0, 255, 0), 2)
        #cv2.putText(image, label, (startX, startY), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)

        # show the output image
        #cv2.imshow("Image", image)
        #cv2.waitKey(0)

        #####JUST INSPECTION- REMOVE

        tfAnnot.xMins.append(xMin)
        tfAnnot.xMaxs.append(xMax)
        tfAnnot.yMins.append(yMin)
        tfAnnot.yMaxs.append(yMax)
        tfAnnot.textLabels.append(label.encode("utf8"))
        tfAnnot.classes.append(config.CLASSES[label])
        tfAnnot.difficult.append(0)

        # increment the total number of examples
        total += 1
      
      features = tf.train.Features(feature=tfAnnot.build())
      example = tf.train.Example(features=features)
      
      writer.write(example.SerializeToString())    
    writer.close()
    print("[INFO] {} examples saved for '{}'".format(total, dType))


In [20]:
create_records(config)

rows in file:  1873
discarded rows:  0
discarded labels:  4
[INFO] processing 'train'...
[INFO] 1400 examples saved for 'train'
[INFO] processing 'test'...
[INFO] 468 examples saved for 'test'


## Faster RCNN model

In this section we are going to train the faster RCNN mode with a pretrained model. The steps are:

- **Download the pretrained** model from Tensorflow model zoo.
- **Create the configuration file** needed to perform finetunning.
- **Launch Tensorboard** monitor training.
- **Start the training** process.
- **Export the model** for later predictions

### Downloading the pretrained model from Tensorflow zoo
Here we used a model that was trained on the COCO dataset taking as the base network **ResNet** architecture. Specifically we downloaded the `faster_rcnn_resnet101_coco` which you can see [here](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md)

In [0]:
!wget http://download.tensorflow.org/models/object_detection/faster_rcnn_resnet101_coco_2018_01_28.tar.gz
!tar -zxvf faster_rcnn_resnet101_coco_2018_01_28.tar.gz
!rm faster_rcnn_resnet101_coco_2018_01_28.tar.gz
!mv faster_rcnn_resnet101_coco_2018_01_28 '/content/invoice/experiments/rcnn/training/'

### Creating the configuration file
This is the file where you can configure things like training steps, the paths to the records, and other configuration for the finetunning process. The file we used, was originally taken from [this tensorflow repo](https://github.com/tensorflow/models/tree/master/research/object_detection/samples/configs) with the name `faster_rcnn_resnet101_pets.config`. We choose the pets and not COCO because is easier, with less configuration to define. Basically the configuration we need to care about is:

- `num_classes`: This value should be 3 as we have this number of classes (header, total and date)
- `num_steps`: This value should be large if you want to perform a good training (between 20.000 and 200.000). For this particular demostration we are going to put 2.500
- `fine_tune_checkpoint`: This is the path to the pretrained model and the value hsould be `/content/invoice/experiments/rcnn/training/faster_rcnn_resnet101_coco_2018_01_28/model.ckpt`
- `input_path` for train input reader: this is the path to the record created previously for training. The value should be `/content/invoice/records/training.record`
- `label_map_path`: The file where the class information is saved. This file was also generated during the creation of TF Records. The value should be `/content/invoice/records/classes.pbtxt`
- `input_path` for test input reader: this is the path to the record created previously for training. The value should be `/content/invoice/records/testing.record`
- `label_map_path` for testing: This is the same file reported for training. The value should be `/content/invoice/records/classes.pbtxt`
- `num_examples`: This is the number of bounding boxes that we have for testing. This number is printed during the creation of TF Records and the values is `468`

In [0]:
path_to_conf = '/content/invoice/experiments/rcnn/training/faster_rcnn_invoice.config'

In [0]:
!rm '/content/invoice/experiments/rcnn/training/faster_rcnn_invoice.config'

In [0]:
%%writefile {path_to_conf}
model {
  faster_rcnn {
    num_classes: 3
    image_resizer {
      keep_aspect_ratio_resizer {
        min_dimension: 600
        max_dimension: 1024
      }
    }
    feature_extractor {
      type: 'faster_rcnn_resnet101'
      first_stage_features_stride: 16
    }
    first_stage_anchor_generator {
      grid_anchor_generator {
        scales: [0.25, 0.5, 1.0, 2.0]
        aspect_ratios: [0.5, 1.0, 2.0]
        height_stride: 16
        width_stride: 16
      }
    }
    first_stage_box_predictor_conv_hyperparams {
      op: CONV
      regularizer {
        l2_regularizer {
          weight: 0.0
        }
      }
      initializer {
        truncated_normal_initializer {
          stddev: 0.01
        }
      }
    }
    first_stage_nms_score_threshold: 0.0
    first_stage_nms_iou_threshold: 0.7
    first_stage_max_proposals: 300
    first_stage_localization_loss_weight: 2.0
    first_stage_objectness_loss_weight: 1.0
    initial_crop_size: 14
    maxpool_kernel_size: 2
    maxpool_stride: 2
    second_stage_box_predictor {
      mask_rcnn_box_predictor {
        use_dropout: false
        dropout_keep_probability: 1.0
        fc_hyperparams {
          op: FC
          regularizer {
            l2_regularizer {
              weight: 0.0
            }
          }
          initializer {
            variance_scaling_initializer {
              factor: 1.0
              uniform: true
              mode: FAN_AVG
            }
          }
        }
      }
    }
    second_stage_post_processing {
      batch_non_max_suppression {
        score_threshold: 0.0
        iou_threshold: 0.6
        max_detections_per_class: 100
        max_total_detections: 300
      }
      score_converter: SOFTMAX
    }
    second_stage_localization_loss_weight: 2.0
    second_stage_classification_loss_weight: 1.0
  }
}

train_config: {
  batch_size: 1
  optimizer {
    momentum_optimizer: {
      learning_rate: {
        manual_step_learning_rate {
          initial_learning_rate: 0.0003
          schedule {
            step: 900000
            learning_rate: .00003
          }
          schedule {
            step: 1200000
            learning_rate: .000003
          }
        }
      }
      momentum_optimizer_value: 0.9
    }
    use_moving_average: false
  }
  gradient_clipping_by_norm: 10.0
  fine_tune_checkpoint: "/content/invoice/experiments/rcnn/training/faster_rcnn_resnet101_coco_2018_01_28/model.ckpt"
  from_detection_checkpoint: true
  load_all_detection_checkpoint_vars: true
  # Note: The below line limits the training process to 200K steps, which we
  # empirically found to be sufficient enough to train the pets dataset. This
  # effectively bypasses the learning rate schedule (the learning rate will
  # never decay). Remove the below line to train indefinitely.
  num_steps: 2500
  data_augmentation_options {
    random_horizontal_flip {
    }
  }
}

train_input_reader: {
  tf_record_input_reader {
    input_path: "/content/invoice/records/training.record"
  }
  label_map_path: "/content/invoice/records/classes.pbtxt"
}

eval_config: {
  metrics_set: "coco_detection_metrics"
  num_examples: 468
}

eval_input_reader: {
  tf_record_input_reader {
    input_path: "/content/invoice/records/testing.record"
  }
  label_map_path: "/content/invoice/records/classes.pbtxt"
  shuffle: false
  num_readers: 1
}

Writing /content/invoice/experiments/rcnn/training/faster_rcnn_invoice.config


### Launching Tensorboard

Tensorboard won't show anything so far because the training process has not started yet, but once the training is running (next step) the Tensorboard should update and start to show the progress

In [0]:
try:
    %load_ext tensorboard
    %tensorboard --logdir '/content/invoice/experiments/rcnn/training/'

except:
    print ("not in colab")

### Training the model

In [0]:
%cd '/content'
!python object_detection/model_main.py --alsologtostderr \
--pipeline_config_path invoice/experiments/rcnn/training/faster_rcnn_invoice.config \
--model_dir invoice/experiments/rcnn/training \
--num_train_steps=2500 \
--sample_1_of_n_eval_examples=1

### Exporting the model
In this step we are going to export the generated model for later predictions

In [0]:
!python object_detection/export_inference_graph.py --input_type image_tensor \
--pipeline_config_path invoice/experiments/rcnn/training/faster_rcnn_invoice.config \
--trained_checkpoint_prefix invoice/experiments/rcnn/training/model.ckpt-2500 \
--output_directory invoice/experiments/rcnn/exported_model

## SSD model

Now we can just train another model, in this case a SSD based on a pretrained model using COCO dataset. The steps are the same as before:

- **Download the pretrained** model from Tensorflow model zoo.
- **Create the configuration file** needed to perform finetunning.
- **Launch Tensorboard** to monitor training.
- **Start the training** process.
- **Export the model** for later predictions

### Downloading the pretarined model

Here we used a model that was trained on the COCO dataset taking the **Inception** as the base network to perform detection. Specifically we downloaded the `ssd_inception_v2_coco` which you can see [here](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md)

In [0]:
!wget http://download.tensorflow.org/models/object_detection/ssd_inception_v2_coco_2018_01_28.tar.gz
!tar -zxvf ssd_inception_v2_coco_2018_01_28.tar.gz
!rm ssd_inception_v2_coco_2018_01_28.tar.gz
!mv ssd_inception_v2_coco_2018_01_28 '/content/invoice/experiments/ssd/training/'

### Creating the configuration file

This is the file where you can configure things like training steps, the paths to the records, and other configuration for the finetunning process. The file we used, was originally taken from [this tensorflow repo](https://github.com/tensorflow/models/tree/master/research/object_detection/samples/configs) with the name `ssd_inception_v2_pets.config`. We choose the pets and not COCO because is easier, with less configuration to define. Basically the configuration we need to care about is:

- `num_classes`: This value should be 3 as we have this number of classes (header, total and date)
- `num_steps`: This value should be large if you want to perform a good training (between 20.000 and 200.000). For this particular demostration we are going to put 2.500
- `fine_tune_checkpoint`: This is the path to the pretrained model and the value should be `/content/invoice/experiments/ssd/training/ssd_inception_v2_coco_2018_01_28/model.ckpt`
- `input_path` for train input reader: this is the path to the record created previously for training. The value should be `/content/invoice/records/training.record`
- `label_map_path`: The file where the class information is saved. This file was also generated during the creation of TF Records. The value should be `/content/invoice/records/classes.pbtxt`
- `input_path` for test input reader: this is the path to the record created previously for training. The value should be `/content/invoice/records/testing.record`
- `label_map_path` for testing: This is the same file reported for training. The value should be `/content/invoice/records/classes.pbtxt`
- `num_examples`: This is the number of bounding boxes that we have for testing. This number is printed during the creation of TF Records and the values is `468`

In [0]:
path_to_conf = '/content/invoice/experiments/ssd/training/ssd_invoices.config'

In [0]:
!rm '/content/invoice/experiments/ssd/training/ssd_invoices.config'

In [24]:
%%writefile {path_to_conf}
model {
  ssd {
    num_classes: 3
    box_coder {
      faster_rcnn_box_coder {
        y_scale: 10.0
        x_scale: 10.0
        height_scale: 5.0
        width_scale: 5.0
      }
    }
    matcher {
      argmax_matcher {
        matched_threshold: 0.5
        unmatched_threshold: 0.5
        ignore_thresholds: false
        negatives_lower_than_unmatched: true
        force_match_for_each_row: true
      }
    }
    similarity_calculator {
      iou_similarity {
      }
    }
    anchor_generator {
      ssd_anchor_generator {
        num_layers: 6
        min_scale: 0.2
        max_scale: 0.95
        aspect_ratios: 1.0
        aspect_ratios: 2.0
        aspect_ratios: 0.5
        aspect_ratios: 3.0
        aspect_ratios: 0.3333
        reduce_boxes_in_lowest_layer: true
      }
    }
    image_resizer {
      fixed_shape_resizer {
        height: 300
        width: 300
      }
    }
    box_predictor {
      convolutional_box_predictor {
        min_depth: 0
        max_depth: 0
        num_layers_before_predictor: 0
        use_dropout: false
        dropout_keep_probability: 0.8
        kernel_size: 3
        box_code_size: 4
        apply_sigmoid_to_scores: false
        conv_hyperparams {
          activation: RELU_6,
          regularizer {
            l2_regularizer {
              weight: 0.00004
            }
          }
          initializer {
            truncated_normal_initializer {
              stddev: 0.03
              mean: 0.0
            }
          }
        }
      }
    }
    feature_extractor {
      type: 'ssd_inception_v3'
      min_depth: 16
      depth_multiplier: 1.0
      conv_hyperparams {
        activation: RELU_6,
        regularizer {
          l2_regularizer {
            weight: 0.00004
          }
        }
        initializer {
          truncated_normal_initializer {
            stddev: 0.1
            mean: 0.0
          }
        }
        batch_norm {
          train: true,
          scale: true,
          center: true,
          decay: 0.9997,
          epsilon: 0.01,
        }
      }
      override_base_feature_extractor_hyperparams: true
    }
    loss {
      classification_loss {
        weighted_sigmoid {
        }
      }
      localization_loss {
        weighted_smooth_l1 {
        }
      }
      hard_example_miner {
        num_hard_examples: 3000
        iou_threshold: 0.99
        loss_type: CLASSIFICATION
        max_negatives_per_positive: 3
        min_negatives_per_image: 0
      }
      classification_weight: 1.0
      localization_weight: 1.0
    }
    normalize_loss_by_num_matches: true
    post_processing {
      batch_non_max_suppression {
        score_threshold: 1e-8
        iou_threshold: 0.6
        max_detections_per_class: 100
        max_total_detections: 100
      }
      score_converter: SIGMOID
    }
  }
}

train_config: {
  batch_size: 24
  optimizer {
    rms_prop_optimizer: {
      learning_rate: {
        exponential_decay_learning_rate {
          initial_learning_rate: 0.004
          decay_steps: 800720
          decay_factor: 0.95
        }
      }
      momentum_optimizer_value: 0.9
      decay: 0.9
      epsilon: 1.0
    }
  }
  fine_tune_checkpoint: "/content/invoice/experiments/ssd/training/ssd_inception_v2_coco_2018_01_28/model.ckpt"
  from_detection_checkpoint: true
  load_all_detection_checkpoint_vars: true
  # Note: The below line limits the training process to 200K steps, which we
  # empirically found to be sufficient enough to train the pets dataset. This
  # effectively bypasses the learning rate schedule (the learning rate will
  # never decay). Remove the below line to train indefinitely.
  num_steps: 2500
  data_augmentation_options {
    random_horizontal_flip {
    }
  }
  data_augmentation_options {
    ssd_random_crop {
    }
  }
}

train_input_reader: {
  tf_record_input_reader {
    input_path: "/content/invoice/records/training.record"
  }
  label_map_path: "/content/invoice/records/classes.pbtxt"
}

eval_config: {
  metrics_set: "coco_detection_metrics"
  num_examples: 468
}

eval_input_reader: {
  tf_record_input_reader {
    input_path: "/content/invoice/records/testing.record"
  }
  label_map_path: "/content/invoice/records/classes.pbtxt"
  shuffle: false
  num_readers: 1
}

Writing /content/invoice/experiments/ssd/training/ssd_invoices.config


### Launching Tensorboard

Tensorboard won't show anything so far because the training process has not started yet, but once the training is running (next step) the Tensorboard should update and start to show the progress

In [0]:
try:
    %load_ext tensorboard
    %tensorboard --logdir '/content/invoice/experiments/ssd/training/'

except:
    print ("not in colab")

### Training the model

In [0]:
%cd '/content'
!python object_detection/model_main.py --alsologtostderr \
--pipeline_config_path invoice/experiments/ssd/training/ssd_invoices.config \
--model_dir invoice/experiments/ssd/training \
--num_train_steps=2500 \
--sample_1_of_n_eval_examples=1


### To export the model

In [0]:
!python object_detection/export_inference_graph.py --input_type image_tensor \
--pipeline_config_path invoice/experiments/ssd/training/ssd_invoices.config \
--trained_checkpoint_prefix invoice/experiments/ssd/training/model.ckpt-2500 \
--output_directory invoice/experiments/ssd/exported_model

## What's Next?

In this notebook we trained two models (faster RCNN and SSD) for object detection on invoices, specifically to detect and localize the **header, the total and the date**. This was the process we followed:
- We configured the environment by installing the Tensorflow Object Detection API
- We downloaded the dataset and created the TF Records that are the standard way the API handles the images and bounding boxes
- Then for each of the two models we:
  * Downloaded a pretrained model (on COCO dataset) from Tensorflow Model Zoo
  * Created a conf define parameters for finetunning
  * Launch Tensorboard to monitor the training process
  * Executed the training process
  * Exported the trained model for later predictions

Now if we are interested in making predictions we can follow the notebook provided in this repository to that end